This notebook addresses the question, "Can we represent a molecule as a graph via a 1D column vector or a 2D matrix of fixed length, with maximum number of atoms n_rows?" Then, can we use this representation to learn neural fingerprints? E.g., can we make an aromatic ring detector? 

Scheme:
feature_matrix = X
for each ligand:
    choose a central atom. this can be the atom (node) that minimizes distance to furthest heavy atom in graph.
    set first row of X to be this central atom
    set next four rows to be the atoms bonded to that centrl atom
        set zeros for rows where row ind > n_bonds of atom
    for each of those atoms:
        repeat. find their neighbors. add to matrix.

algorithm: breadth-first search:
1. create networkx graph based on molecule
2. find "central" atom (different strategies)
3. define atom matrix of size (1+4+4*3^(L-1)) x (n_features_per_atom)
4. start atom queue q
5. central_atom.layer = 0; central_atom.row_idx = 0;
6. q.enqueue(central_atom)
7. define adjacency matrix of size (1+4+4*3^(L-1)) x 4

def get_row_idx(curr_layer, prev_row_idx, curr_neighbor_idx):
    if curr_layer == 0:
        return(0)
    if curr_layer == 1:
        row_idx = 1 + curr_neighbor_idx
    if layer == 2:
        last_max = 5
        row_idx = last_max + (3*(prev_row_idx-last_max)) + curr_neighbor_idx
    if layer > 2:
        last_max = 5 + 4*3^(curr_layer-2) 
        row_idx = last_max + 3*(prev_row_idx-last_max) + curr_neighbor_idx
    return(row_idx)
    

while q.is_not_empty():
    a = q.dequeue()
    a.visited = True
    for n_idx, n in enumerate(a.neighbors()):
        if not n.visited:
            row_idx = c
            n.layer = a.layer + 1
            row_idx = get_row_idx(n.layer, a.row_idx, n_idx)
            n.row_idx = row_idx
            adj_matrix[a.row_idx][n_idx] = n.row_idx
            atom_matrix[row_idx][elem_to_idx[n.elem]] = 1

input_matrix = tf.concat([atom_matrix, atom_matrix[adj_matrix[:,0]], atom_matrix[adj_matrix[:,1]], atom_matrix[adj_matrix[:,2]], atom_matrix[adj_matrix[:,3]]

neural net:
h1 = relu([tf.zeros([n_features_per_atom, 4]) * input_matrix + bias))
h1_conc = tf.concat([h1, h1[adj_matrix[:,0], ..., h1[adj_matrix[:,3])

repeat h1 to get h2


dihedral predictor pseudocode:

get bonds for molecule
create networkx graph out of molecule (use atom indices)

for each edge:
   for neighbor_i in atom_i.neighbors():
       if neighbor_i == atom_j: continue
       for neighbor_j in atom_j.neighbors():
           if neighbor_j == atom_i: continue
           dihedrals.append((neighbor_i, atom_i, neighbor_j, atom_j))
           check to make sure (atom_j, neighbor_j, atom_i, neighbor_i)) not already in list

for dihedral in dihedrals:
    angle =  rdMolTransforms.GetDihedralDeg(c, 0,1,2,3)


In [1]:
from model import DCGAN
gan = DCGAN(n_layers=2, batch_size=64,
                                 n_atom_types=75,
                                 max_n_atoms=200,
                                 max_valence=6,
                                 n_tasks=1,
                                 learning_rate=1e-3,
                                 L_list = [50, 50, 50, 50],
                                 epsilon=1e-8,
                                 beta1=0.9,
                                 dropout=1.0)

layer_idx: 0
layer_idx: 1


ValueError: Shape must be rank 2 but is rank 3 for 'generator/MatMul_2' (op: 'MatMul') with input shapes: [64,200,200], [12800,50].

In [1]:
featurizer = dc.feat.AdjacencyFingerprint(n_atom_types=23, max_n_atoms=200)

NameError: name 'dc' is not defined

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolTransforms
import os
import fnmatch
import numpy as np
import deepchem as dc
from scipy.sparse import csr_matrix

In [2]:
def get_torsions_angles(mol):
    torsion_tuples = []
    for bond in mol.GetBonds():
        atom_i = bond.GetBeginAtom()
        atom_j = bond.GetEndAtom()
        if atom_i.IsInRing() or atom_j.IsInRing():
            continue
        for neighbor_i in atom_i.GetNeighbors():
            if neighbor_i.GetIdx() == atom_j.GetIdx():
                continue
            
            for neighbor_j in atom_j.GetNeighbors():
                if neighbor_j.GetIdx() == atom_i.GetIdx():
                    continue
                torsion_tuple = (neighbor_i.GetIdx(), atom_i.GetIdx(), atom_j.GetIdx(), neighbor_j.GetIdx())
                reverse_torsion_tuple = (neighbor_j.GetIdx(), atom_j.GetIdx(), atom_i.GetIdx(), neighbor_i.GetIdx())
                if torsion_tuple not in torsion_tuples and reverse_torsion_tuple not in torsion_tuples:
                    torsion_tuples.append(torsion_tuple)
    c = mol.GetConformer(0)
    torsions = []
    torsion_matrix = np.zeros((250,1))
    torsion_indices = np.zeros((250,200,4)).astype(np.uint8)
    for i, torsion_tuple in enumerate(torsion_tuples):
        torsion_matrix[i] = np.cos(rdMolTransforms.GetDihedralRad(c, *torsion_tuple))
        torsion_indices[i][torsion_tuple[0]][0] = 1
        torsion_indices[i][torsion_tuple[1]][1] = 1
        torsion_indices[i][torsion_tuple[2]][2] = 1
        torsion_indices[i][torsion_tuple[3]][3] = 1
    return((torsion_indices, csr_matrix(torsion_matrix)))
                

In [3]:
def featurize_mols(mol_files):
    featurizer = AdjacencyFingerprint(max_n_atoms=200)
    features = []
    for mol_file in mol_files:
        mol = Chem.MolFromMol2File(mol_file)
        if mol is None:
            features.append(None)
            continue
        torsions = get_torsions_angles(mol)
        graph_feat = featurizer.featurize([mol])[0]
        features.append((mol_file, torsions, graph_feat))
    return(features)

In [4]:
from deepchem.feat.graph_features import ConvMolFeaturizer
from deepchem.feat.adjacency_fingerprints import AdjacencyFingerprint

In [6]:
import pickle
feature_file = "../../dihedral_features_pdbbind.pkl"
if not os.path.exists(feature_file):
#if 1== 1:
    pdbbind_dir = "/home/evan/Documents/deep_docking/datasets/v2015/"
    def find_files(directory, pattern):
        for root, dirs, files in os.walk(directory):
            for basename in files:
                if fnmatch.fnmatch(basename, pattern):
                    filename = os.path.join(root, basename)
                    yield filename
    ligand_files = [f for f in find_files(pdbbind_dir, "*ligand.mol2")]
    features = featurize_mols(ligand_files)
    with open(feature_file, "wb") as f:
        pickle.dump(features, f, protocol=2)
else:
    with open(feature_file, "rb") as f:
        features = pickle.load(f)

In [7]:
features = [f for f in features if f is not None and len(np.where(f[1][1].toarray() == 0)[0]) < 249]

In [8]:
len(features)

7258

In [ ]:
model.fit(features[0:6976], features[7000:7256], n_epochs=10000)

Training epoch 0
train_acc: 0.579399929805
test_acc: 0.481832385063
Training epoch 100
train_acc: 0.139934486218
test_acc: 0.203423684835
Training epoch 200
train_acc: 0.104905637489
test_acc: 0.187368488312
Training epoch 300
train_acc: 0.0899346297099
test_acc: 0.182623833418


In [25]:
x

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
len(features)

In [10]:
features[9][1][1]

array([[-0.04800897],
       [-0.07369911],
       [-0.85061992],
       [-0.25623746],
       [-0.99060946],
       [ 0.32041239],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0

In [ ]:
features[0][2].atom_features